<a href="https://colab.research.google.com/github/babybeelz20/Big-Data-Midterm/blob/main/Recommendation_Systems_%E2%80%93_Frequent_Pattern_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Market Basket Analysis

In [1]:
%%capture
!sudo apt-get update --fix-missing

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
#!wget -q https://downloads.apache.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

!mv spark-3.0.0-bin-hadoop3.2.tgz sparkkk
!tar xf sparkkk
!pip install -q findspark

In [2]:
!pip install spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName('fpgrowth') \
    .getOrCreate()

spark   

In [4]:
# To be able to use your data stored in your Google Drive you first need to mount your Google Drive so you can load and save files to it. 
from google.colab import drive
drive.mount('/content/gdrive')
#You'll need to put in a token which Google will generate for you as soon as you click on the link

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
#Read the dataset
import pandas as pd
data = pd.read_csv('/content/gdrive/MyDrive/Google Drive/market-basket.csv', sep=';')
data.head()

<ipython-input-5-ad4e14bc6d77>:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/gdrive/MyDrive/Google Drive/market-basket.csv', sep=';')


,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,"2,55",17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 08:26,"2,75",17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom


In [6]:
data

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,"2,55",17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 08:26,"2,75",17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
...,...,...,...,...,...,...,...
522059,581587,PACK OF 20 SPACEBOY NAPKINS,12,09.12.2011 12:50,"0,85",12680.0,France
522060,581587,CHILDREN'S APRON DOLLY GIRL,6,09.12.2011 12:50,"2,1",12680.0,France
522061,581587,CHILDRENS CUTLERY DOLLY GIRL,4,09.12.2011 12:50,"4,15",12680.0,France
522062,581587,CHILDRENS CUTLERY CIRCUS PARADE,4,09.12.2011 12:50,"4,15",12680.0,France


In [7]:
from google.colab import files
from pyspark.sql import functions as F
from pyspark.ml.fpm import FPGrowth
import pandas

In [8]:
sparkdata = spark.read.options(header='True',inferSchema='True',delimiter=';') \
  .csv('/content/gdrive/MyDrive/Google Drive/market-basket.csv')

In [9]:
sparkdata

DataFrame[BillNo: string, Itemname: string, Quantity: int, Date: string, Price: string, CustomerID: int, Country: string]

In [10]:
basketdata = sparkdata.dropDuplicates(['BillNo', 'Itemname']).sort('BillNo')
basketdata = basketdata.groupBy("BillNo").agg(F.collect_list("Itemname")).sort('BillNo')

In [11]:
#Frequent Pattern Growth – FP Growth is a method of mining frequent itemsets
fpGrowth = FPGrowth(itemsCol="collect_list(Itemname)", minSupport=0.006, minConfidence=0.006) 
model = fpGrowth.fit(basketdata)


In [12]:
# Display frequent itemsets.
model.freqItemsets.show()

+--------------------+----+
|               items|freq|
+--------------------+----+
|[JUMBO BAG VINTAG...| 470|
|[JUMBO BAG VINTAG...| 130|
|[JUMBO BAG VINTAG...| 135|
|[JUMBO BAG VINTAG...| 136|
|[JUMBO BAG VINTAG...| 183|
|[JUMBO BAG VINTAG...| 180|
|[JUMBO BAG VINTAG...| 132|
|[JUMBO BAG VINTAG...| 319|
|[JUMBO BAG VINTAG...| 130|
|[JUMBO BAG VINTAG...| 130|
|[JUMBO BAG VINTAG...| 154|
|[JAM JAR WITH PIN...| 333|
|[JAM JAR WITH PIN...| 153|
|[JAM JAR WITH PIN...| 132|
|[STRAWBERRY   PIC...| 245|
|[STRAWBERRY FAIRY...| 198|
|[CAMOUFLAGE LED T...| 160|
|[PINK LOVE HEART ...| 133|
|[WHITE HANGING HE...|2202|
|[BIG DOUGHNUT FRI...| 198|
+--------------------+----+
only showing top 20 rows



In [13]:
items = model.freqItemsets

In [14]:
# Display generated association rules.
model.associationRules.show()

+--------------------+--------------------+-------------------+------------------+
|          antecedent|          consequent|         confidence|              lift|
+--------------------+--------------------+-------------------+------------------+
|[HOT WATER BOTTLE...|[LOVE HOT WATER B...| 0.3009079118028534|14.018426007280029|
|[HOT WATER BOTTLE...|[CHOCOLATE HOT WA...| 0.3450064850843061| 8.940042447824549|
|[HOT WATER BOTTLE...|[PAPER CHAIN KIT ...|0.21660181582360571| 4.105201343995425|
|[HOT WATER BOTTLE...|[GARDENERS KNEELI...|0.22568093385214008| 5.402128254186642|
|[HOT WATER BOTTLE...|[SCOTTIE DOG HOT ...|0.25551232166018156| 7.292705433629135|
|[HOT WATER BOTTLE...|[HAND WARMER RED ...|0.17250324254215305| 9.484613561397618|
|[HOT WATER BOTTLE...|[HAND WARMER OWL ...| 0.2127107652399481|7.1000821377395935|
|[HOT WATER BOTTLE...|[HOT WATER BOTTLE...| 0.2788586251621271| 9.409523982690281|
|[HOT WATER BOTTLE...|[HOT WATER BOTTLE...| 0.3411154345006485|11.673907831891862|
|[HO

In [15]:
rules = model.associationRules

In [16]:
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(basketdata).show()

+------+----------------------+--------------------+
|BillNo|collect_list(Itemname)|          prediction|
+------+----------------------+--------------------+
|536365|  [KNITTED UNION FL...|[JUMBO BAG RED RE...|
|536366|  [HAND WARMER UNIO...|[HAND WARMER RED ...|
|536367|  [BOX OF VINTAGE J...|[POPPY'S PLAYHOUS...|
|536368|  [YELLOW COAT RACK...|[JAM JAR WITH PIN...|
|536369|  [BATH BUILDING BL...|[HOME BUILDING BL...|
|536370|  [SPACEBOY LUNCH B...|[WOODLAND CHARLOT...|
|536371|  [PAPER CHAIN KIT ...|[JUMBO BAG VINTAG...|
|536372|  [HAND WARMER UNIO...|[HAND WARMER RED ...|
|536373|  [GLASS STAR FROST...|[HEART OF WICKER ...|
|536374|  [VICTORIAN SEWING...|                  []|
|536375|  [SAVE THE PLANET ...|[HEART OF WICKER ...|
|536376|  [RED HANGING HEAR...|[LOVE HOT WATER B...|
|536377|  [HAND WARMER RED ...|[HAND WARMER RED ...|
|536378|  [PACK OF 60 PINK ...|[JUMBO BAG RED RE...|
|536380|  [JAM MAKING SET P...|[JAM JAR WITH PIN...|
|536381|  [ZINC WILLIE WINK...|[SMALL HEART ME

In [17]:
transformed = model.transform(basketdata)

In [18]:
# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = items.select("*").toPandas()
result_pdf.head()

,items,freq
0,[JUMBO BAG VINTAGE CHRISTMAS],470
1,"[JUMBO BAG VINTAGE CHRISTMAS, SET OF 20 VINTAG...",130
2,"[JUMBO BAG VINTAGE CHRISTMAS, JUMBO BAG ALPHABET]",135
3,"[JUMBO BAG VINTAGE CHRISTMAS, JUMBO BAG VINTAG...",136
4,"[JUMBO BAG VINTAGE CHRISTMAS, JUMBO BAG RED RE...",183


In [19]:
result_pdf.to_excel('result_pdfItemsFreq.xlsx')

In [20]:
rules_pdf = rules.select("*").toPandas()
rules_pdf.head()

,antecedent,consequent,confidence,lift
0,[HOT WATER BOTTLE KEEP CALM],[LOVE HOT WATER BOTTLE],0.300908,14.018426
1,[HOT WATER BOTTLE KEEP CALM],[CHOCOLATE HOT WATER BOTTLE],0.345006,8.940042
2,[HOT WATER BOTTLE KEEP CALM],[PAPER CHAIN KIT 50'S CHRISTMAS],0.216602,4.105201
3,[HOT WATER BOTTLE KEEP CALM],[GARDENERS KNEELING PAD KEEP CALM],0.225681,5.402128
4,[HOT WATER BOTTLE KEEP CALM],[SCOTTIE DOG HOT WATER BOTTLE],0.255512,7.292705


In [21]:
rules_pdf.to_excel('rules_pdfAnteConseConfLift.xlsx')

In [22]:
transformed_pdf = transformed.select("*").toPandas()
transformed_pdf.head()

,BillNo,collect_list(Itemname),prediction
0,536365,"[KNITTED UNION FLAG HOT WATER BOTTLE, SET 7 BA...","[JUMBO BAG RED RETROSPOT, REGENCY CAKESTAND 3 ..."
1,536366,"[HAND WARMER UNION JACK, HAND WARMER RED POLKA...","[HAND WARMER RED LOVE HEART, HAND WARMER RED R..."
2,536367,"[BOX OF VINTAGE JIGSAW BLOCKS, FELTCRAFT PRINC...","[POPPY'S PLAYHOUSE LIVINGROOM, POPPY'S PLAYHOU..."
3,536368,"[YELLOW COAT RACK PARIS FASHION, BLUE COAT RAC...","[JAM JAR WITH PINK LID, SMALL HEART MEASURING ..."
4,536369,[BATH BUILDING BLOCK WORD],[HOME BUILDING BLOCK WORD]


In [23]:
transformed_pdf.to_excel('transformed_pdfSalesTransactionIDCollectListPred.xlsx')